# Tutorial Práctico: MLOps Básico en la Nube

¡Hola! Bienvenido a esta guía paso a paso para construir tu primer flujo de trabajo de Machine Learning en la nube. No necesitas tener experiencia previa con la nube o GCP. Te guiaremos desde cero. Al final de este tutorial, habrás aprendido a:

1.  **Configurar tu entorno de trabajo** en la nube.
2.  **Crear un Data Lake** para almacenar tus datos.
3.  **Entrenar y guardar un modelo** de Machine Learning.
4.  **Desplegar tu modelo** como un servicio web accesible a través de internet.

Todo esto se realizará usando los servicios de Google Cloud Platform (GCP), una de las plataformas de computación en la nube más populares.

## 0. Configuración Inicial del Entorno (Paso a Paso)

Antes de empezar con el Machine Learning, necesitamos preparar nuestro entorno de trabajo. Si ya tienes el SDK de Google Cloud instalado, puedes saltar esta sección.

### 0.1. Instalar el SDK de Google Cloud

El SDK (Software Development Kit) de Google Cloud es un conjunto de herramientas que te permiten interactuar con los servicios de GCP desde tu terminal. Lo usaremos para crear recursos y desplegar nuestro modelo. 

* **Descarga:** Ve a la [página de descarga del SDK](https://cloud.google.com/sdk/docs/install) y sigue las instrucciones para tu sistema operativo (Windows, macOS, Linux).
* **Configuración:** Una vez instalado, abre una terminal o línea de comandos y ejecuta el siguiente comando para inicializar el SDK y autenticarte con tu cuenta de Google.

```bash
gcloud init
```
Este comando te guiará para elegir una cuenta de Google y un proyecto de GCP. Si no tienes un proyecto, te preguntará si quieres crear uno. Elige la opción de crear un proyecto nuevo y dale un nombre único (por ejemplo, `mlops-curso-posgrado-tu-nombre`).

---

### 0.2. Creación y Configuración del Proyecto

Un error común es no tener un proyecto creado o no configurar `gcloud` para que apunte al proyecto correcto. Vamos a asegurarnos de que esto esté bien configurado.

#### **1. Crea un nuevo proyecto (si no lo hiciste con `gcloud init`):**

Elige un `PROJECT_ID` que sea único a nivel global y solo contenga letras, números y guiones. Es una buena práctica usar un prefijo para tus proyectos. Si el comando falla, asegúrate de que el ID sea único y de que la cuenta activa tenga los permisos necesarios.

* **Para Terminales Bash (Linux/macOS/Git Bash):**
```bash
PROJECT_ID="mlops-curso-test-tu-nombre"
gcloud projects create $PROJECT_ID
```

* **Para Terminal de Windows (CMD):**
```bat
set PROJECT_ID=mlops-curso-test-tu-nombre
gcloud projects create %PROJECT_ID%
```

**Nota:** `gcloud` confirmará que el proyecto se ha creado. Puede tardar unos minutos en aparecer en tu lista de proyectos. Puedes verificarlo con `gcloud projects list`.

#### **2. Configura `gcloud` para usar tu proyecto:**

Una vez que el proyecto esté creado, ejecuta el siguiente comando para que todas las operaciones de `gcloud` apunten a tu nuevo proyecto.

* **Para Terminales Bash (Linux/macOS/Git Bash):**
```bash
gcloud config set project $PROJECT_ID
echo "El proyecto $PROJECT_ID ha sido configurado como el proyecto por defecto."
```

* **Para Terminal de Windows (CMD):**
```bat
gcloud config set project %PROJECT_ID%
echo El proyecto %PROJECT_ID% ha sido configurado como el proyecto por defecto.
```

---

### 0.3. Habilitar Facturación y APIs de GCP

**Nota Importante:** Para poder usar la mayoría de los servicios de GCP, incluso en el nivel gratuito, necesitas tener una cuenta de facturación vinculada a tu proyecto. Si ves un error de "Billing not found", ve a la [página de facturación de GCP](https://console.cloud.google.com/billing) y vincula una cuenta.

Una vez que la facturación esté configurada, puedes habilitar los APIs necesarios para este tutorial:

```bash
gcloud services enable storage.googleapis.com
gcloud services enable run.googleapis.com
gcloud services enable cloudbuild.googleapis.com
```

---

### 0.4. Instalación de las Librerías de Python

Los scripts de Python en este tutorial requieren que varias librerías estén instaladas. El error `ModuleNotFoundError` significa que Python no puede encontrar una de estas librerías. Para evitarlo, ejecuta el siguiente comando en tu terminal para instalar todas las dependencias necesarias:

```bash
pip install google-cloud-storage pandas scikit-learn fastapi uvicorn gunicorn pydantic
```

---

### 0.5. Autenticación de las Librerías de Python (ADC)

Aunque ya te has autenticado con `gcloud auth login`, las librerías de Python necesitan una autenticación separada, llamada **Application Default Credentials (ADC)**. Si no haces este paso, tus scripts de Python no podrán conectarse a los servicios de GCP y verás un error de "DefaultCredentialsError".

Ejecuta el siguiente comando en tu terminal para configurar las credenciales de la aplicación:

```bash
gcloud auth application-default login
```

Este comando abrirá una ventana en tu navegador para que inicies sesión. Usa la misma cuenta de Google que estás usando para tu proyecto de GCP. Una vez que inicies sesión, las credenciales se guardarán automáticamente para que los scripts de Python puedan usarlas.

---

## 1. Creación del Data Lake en Google Cloud Storage

Un **Data Lake** es un lugar para almacenar grandes cantidades de datos sin procesar. Piensa en él como un disco duro muy grande en la nube. En GCP, usamos **Google Cloud Storage (GCS)** para esto.

### 1.1. Crear un 'Bucket' de Almacenamiento

En GCS, el contenedor principal para archivos se llama 'bucket'. El nombre del bucket debe ser único en todo el mundo.

* **Para Terminales Bash (Linux/macOS/Git Bash):**
```bash
BUCKET_NAME="mlops-data-lake-tutorial-$(date +%s)"
REGION="us-central1" # Puedes elegir la región más cercana a ti
gsutil mb -p $PROJECT_ID -l $REGION gs://$BUCKET_NAME
echo "Bucket creado: gs://$BUCKET_NAME"
```

* **Para Terminal de Windows (CMD):**
```bat
rem Para generar un nombre de bucket único, debes hacerlo manualmente. 
rem Por ejemplo, "mlops-data-lake-tutorial-2023-10-27"
set BUCKET_NAME=mlops-data-lake-tutorial-tu-nombre-unico
set REGION=us-central1
gsutil mb -p %PROJECT_ID% -l %REGION% gs://%BUCKET_NAME%
echo Bucket creado: gs://%BUCKET_NAME%
```

**Explicación:**
- `BUCKET_NAME`: Creamos una variable con un nombre único para nuestro bucket.
- `gsutil mb`: Este comando crea (`mb` de *make bucket*) el bucket en GCS. 
- `-p $PROJECT_ID`: Le pasamos explícitamente el ID del proyecto que configuramos antes para que sepa dónde crearlo.

### 1.2. Simular Carga de Datos

Vamos a simular que recibimos un dataset. Usaremos el famoso dataset de Iris de scikit-learn y lo guardaremos en nuestro Data Lake.

```python
import pandas as pd
from sklearn import datasets

# 1. Cargar el dataset de Iris en memoria
iris = datasets.load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

# 2. Guardar el dataset en un archivo local
file_path = "iris_raw.csv"
df.to_csv(file_path, index=False)

print(f"Dataset guardado localmente en {file_path}")
print("Primeras 5 filas del dataset:")
print(df.head())
```

Ahora, subimos este archivo a nuestro bucket. Creamos una carpeta llamada `raw/` para indicar que son datos "crudos" o sin procesar.

* **Para Terminales Bash (Linux/macOS/Git Bash):**
```bash
# Reemplaza con el nombre de tu bucket de la celda anterior
BUCKET_NAME="mlops-data-lake-tutorial-1756515865"
gsutil cp iris_raw.csv gs://$BUCKET_NAME/raw/iris_data/
echo "Archivo subido. Verificando el contenido del bucket..."
gsutil ls gs://$BUCKET_NAME/raw/iris_data/
```

* **Para Terminal de Windows (CMD):**
```bat
set BUCKET_NAME=mlops-data-lake-tutorial-tu-nombre-unico
gsutil cp iris_raw.csv gs://%BUCKET_NAME%/raw/iris_data/
echo Archivo subido. Verificando el contenido del bucket...
gsutil ls gs://%BUCKET_NAME%/raw/iris_data/
```
**Explicación:**
- `gsutil cp`: Copia un archivo desde tu máquina local a GCS.

---

## 2. Preparación de los Datos para el Modelo

En un proyecto real, se procesarían los datos para limpiarlos y prepararlos para el modelo. Aquí, haremos un pequeño cambio a las columnas y guardaremos el resultado en una nueva carpeta, `curated/`, que contiene datos 'curados' o listos para usar.

In [ ]:
import pandas as pd
from google.cloud import storage

# 1. Define tu bucket y las rutas de los archivos
BUCKET_NAME = "mlops-data-lake-tutorial-1756515865"  # Reemplaza con el nombre de tu bucket
RAW_FILE_PATH = "raw/iris_data/iris_raw.csv"
CURATED_FILE_PATH = "curated/iris_data/iris_processed.csv"

# 2. Descargar el archivo crudo desde GCS
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)
blob = bucket.blob(RAW_FILE_PATH)
blob.download_to_filename("iris_raw.csv")

# 3. Leer y procesar los datos con Pandas
df_raw = pd.read_csv("iris_raw.csv")
df_processed = df_raw.rename(columns={
    'sepal length (cm)': 'sepal_length',
    'sepal width (cm)': 'sepal_width',
    'petal length (cm)': 'petal_length',
    'petal width (cm)': 'petal_width'
})

# 4. Guardar los datos procesados localmente
df_processed.to_csv("iris_processed.csv", index=False)

# 5. Subir el archivo procesado a la carpeta 'curated' en GCS
blob = bucket.blob(CURATED_FILE_PATH)
blob.upload_from_filename("iris_processed.csv")

print("Datos procesados y subidos a la capa 'curated' del Data Lake.")


---

## 3. Entrenamiento y Versionado del Modelo

En MLOps, es crucial guardar los modelos de manera organizada y versionada. Así podemos saber qué modelo se usó para una predicción específica. Entrenaremos un modelo `RandomForestClassifier` y lo guardaremos con una etiqueta de versión.

In [ ]:
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from google.cloud import storage
import pandas as pd

# 1. Define tu bucket y el archivo de datos procesados
BUCKET_NAME = "mlops-data-lake-tutorial-1756515865" # Reemplaza con el nombre de tu bucket
CURATED_FILE_PATH = "curated/iris_data/iris_processed.csv"

# 2. Descargar los datos procesados para el entrenamiento
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)
blob = bucket.blob(CURATED_FILE_PATH)
blob.download_to_filename("iris_processed.csv")

# 3. Preparar los datos para el modelo
df_curated = pd.read_csv("iris_processed.csv")
X = df_curated.drop(columns=['target'])
y = df_curated['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Entrenar el modelo de Machine Learning
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 5. Guardar el modelo entrenado localmente
model_filename = 'iris_model.joblib'
joblib.dump(model, model_filename)

print("Modelo entrenado y guardado localmente.")

# 6. Subir el modelo a una carpeta versionada en GCS
VERSION = 'v1.0' # Esta es la versión de nuestro modelo
MODEL_GCS_PATH = f"models/{VERSION}/{model_filename}"
blob = bucket.blob(MODEL_GCS_PATH)
blob.upload_from_filename(model_filename)

print(f"Modelo versionado subido a: gs://{BUCKET_NAME}/{MODEL_GCS_PATH}")


---

## 4. Despliegue del Modelo en Cloud Run

Ahora que tenemos nuestro modelo guardado, lo haremos accesible como un servicio web. Usaremos **Docker** para empaquetar nuestra aplicación y **Cloud Run** para ejecutarla de manera escalable y sin servidor. 

### 4.1. Crear los Archivos de la Aplicación

Necesitamos tres archivos en una misma carpeta en tu máquina local. Estos archivos le dirán a Docker cómo crear una imagen de nuestra aplicación y qué debe hacer la aplicación.

**Archivo 1: `main.py`** (la lógica de la API)
Copia este código y guárdalo como `main.py`.

```python
# main.py
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import os
from google.cloud import storage

app = FastAPI()

# Descargar el modelo desde GCS cuando se inicie la aplicación
BUCKET_NAME = os.environ.get('BUCKET_NAME')
MODEL_PATH = os.environ.get('MODEL_PATH')
LOCAL_MODEL_PATH = '/tmp/iris_model.joblib'

if not os.path.exists(LOCAL_MODEL_PATH):
    client = storage.Client()
    bucket = client.bucket(BUCKET_NAME)
    blob = bucket.blob(MODEL_PATH)
    blob.download_to_filename(LOCAL_MODEL_PATH)

model = joblib.load(LOCAL_MODEL_PATH)

# Define la estructura de datos que esperamos para la predicción
class Item(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

# Un endpoint para verificar que el servicio está funcionando
@app.get("/health")
def health_check():
    return {"status": "ok"}

# El endpoint principal para realizar predicciones
@app.post("/predict")
def predict(item: Item):
    data = [[item.sepal_length, item.sepal_width, item.petal_length, item.petal_width]]
    prediction = model.predict(data)
    return {"prediction": int(prediction[0])}
```

**Archivo 2: `requirements.txt`** (las dependencias de Python)
Copia este código y guárdalo como `requirements.txt`.

```text
fastapi
uvicorn
scikit-learn
joblib
pandas
google-cloud-storage
pydantic
gunicorn
```

**Archivo 3: `Dockerfile`** (las instrucciones para Docker)
Copia este código y guárdalo como `Dockerfile` (sin extensión).

```dockerfile
FROM python:3.9-slim
WORKDIR /app
COPY requirements.txt ./requirements.txt
RUN pip install --no-cache-dir -r requirements.txt
COPY main.py .
CMD exec gunicorn --bind :$PORT --workers 1 --worker-class uvicorn.workers.UvicornWorker main:app
```

### 4.2. Construir la Imagen de Docker y Desplegarla

Ahora, desde tu terminal, navega a la carpeta donde guardaste los archivos anteriores y ejecuta los siguientes comandos. No olvides reemplazar las variables con tus valores.

* **Para Terminales Bash (Linux/macOS/Git Bash):**
```bash
# 1. Define tus variables (reemplaza con tu información)
PROJECT_ID="el-id-de-tu-proyecto"
BUCKET_NAME="mlops-data-lake-tutorial-1756515865"
MODEL_PATH="models/v1.0/iris_model.joblib"
SERVICE_NAME="iris-predictor"
REGION="us-central1"

# 2. Construir la imagen de Docker y subirla a Google Container Registry
gcloud builds submit . --tag gcr.io/$PROJECT_ID/$SERVICE_NAME

# 3. Desplegar el servicio en Cloud Run
gcloud run deploy $SERVICE_NAME \
    --image gcr.io/$PROJECT_ID/$SERVICE_NAME \
    --platform managed \
    --region $REGION \
    --no-allow-unauthenticated \
    --set-env-vars BUCKET_NAME=$BUCKET_NAME,MODEL_PATH=$MODEL_PATH
```

* **Para Terminal de Windows (CMD):**
```bat
rem 1. Define tus variables (reemplaza con tu información)
set PROJECT_ID=el-id-de-tu-proyecto
set BUCKET_NAME=mlops-data-lake-tutorial-tu-nombre-unico
set MODEL_PATH=models/v1.0/iris_model.joblib
set SERVICE_NAME=iris-predictor
set REGION=us-central1

rem 2. Construir la imagen de Docker y subirla a Google Container Registry
gcloud builds submit . --tag gcr.io/%PROJECT_ID%/%SERVICE_NAME%

rem 3. Desplegar el servicio en Cloud Run (el comando es el mismo)
gcloud run deploy %SERVICE_NAME% ^
    --image gcr.io/%PROJECT_ID%/%SERVICE_NAME% ^
    --platform managed ^
    --region %REGION% ^
    --no-allow-unauthenticated ^
    --set-env-vars BUCKET_NAME=%BUCKET_NAME%,MODEL_PATH=%MODEL_PATH%
```

**Explicación:**
- `gcloud builds submit`: Toma tu `Dockerfile` y tu código, construye la imagen del contenedor y la sube al registro de imágenes de Google.
- **(Corrección):** El punto (`.`) indica que el código fuente para la construcción está en el directorio actual. Esto corrige el error de "Dockerfile required".
- `gcloud run deploy`: Toma la imagen del contenedor y la despliega en Cloud Run, creando un servicio web. Le pasamos las variables de entorno para que la aplicación sepa dónde encontrar el modelo en nuestro Data Lake.

---

### 4.3. Probar el Endpoint de Predicción

Una vez que el despliegue esté completo, Cloud Run te proporcionará una URL para tu servicio. Vamos a obtenerla y a hacer una prueba.

* **Para Terminales Bash (Linux/macOS/Git Bash):**
```bash
# 1. Obtener la URL del servicio desplegado
SERVICE_URL=$(gcloud run services describe $SERVICE_NAME --platform managed --region $REGION --format 'value(status.url)')
echo "URL del servicio: $SERVICE_URL"

# 2. Obtener un token de autenticación (necesario para servicios no públicos)
TOKEN=$(gcloud auth print-identity-token)

# 3. Hacer una solicitud POST para obtener una predicción
curl -X POST "$SERVICE_URL/predict" \
    -H "Authorization: Bearer $TOKEN" \
    -H "Content-Type: application/json" \
    -d '{"sepal_length": 5.1, "sepal_width": 3.5, "petal_length": 1.4, "petal_width": 0.2}'
```

* **Para Terminal de Windows (CMD):**
```bat
rem 1. Obtener la URL del servicio desplegado
rem Este comando de una sola línea usa 'for' para capturar la URL y la asigna a la variable SERVICE_URL
rem Opcional gcloud run services describe %SERVICE_NAME% --platform managed --region %REGION% --format "value(status.url)" y crear SERVICE_URL copiando la URL
for /f "delims=" %%a in ('gcloud run services describe %SERVICE_NAME% --platform managed --region %REGION% --format "value(status.url)"') do set "SERVICE_URL=%%a"
echo URL del servicio: %SERVICE_URL%

rem 2. Obtener un token de autenticación (necesario para servicios no públicos)
rem Similar al anterior, este comando captura el token y lo asigna a la variable TOKEN
rem Opcional gcloud auth print-identity-token y crear TOKEN copiando la Url

for /f "delims=" %%a in ('gcloud auth print-identity-token') do set "TOKEN=%%a"

rem 3. Hacer una solicitud POST para obtener una predicción
curl -X POST "%SERVICE_URL%/predict" -H "Authorization: Bearer %TOKEN%" -H "Content-Type: application/json" -d "{\"sepal_length\": 5.1, \"sepal_width\": 3.5, \"petal_length\": 1.4, \"petal_width\": 0.2}"
```

---

## Conclusión

¡Felicidades! Has completado tu primer pipeline de MLOps en GCP. Desde el almacenamiento de datos en un Data Lake, pasando por el entrenamiento del modelo, hasta su despliegue como un servicio web escalable. 

Este flujo de trabajo demuestra los principios clave de MLOps: **reproducibilidad**, **automatización** y **escalabilidad**, que son cruciales para llevar modelos de Machine Learning a producción de manera efectiva.